In [2]:
# Batch Import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


## Import CSV and EDA

In [5]:
#Import Housing data from https://www.redfin.com/blog/data-center
df=pd.read_csv('./assets/Housing_Data_3.csv')

print(df.shape)

FileNotFoundError: File b'./assets/Housing_Data_3.csv' does not exist

In [ ]:
df.head(3)

In [ ]:
#fix messy column names 
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
# df.columns=[x.lower() for x in df.columns]

In [ ]:
#56 unique districts same as PSAs
df.region.nunique()

In [ ]:
#null value check
df.isnull().sum()

In [ ]:
df.dropna(inplace=True)
df.shape # 8/2012 dropped

In [ ]:
# check no more null values
df.isnull().sum().sum()

In [ ]:
df.info()

In [ ]:
# Differentiate numerical and categorical features for further EDA
ob_features = df.select_dtypes(include = ["object"]).columns
num_features = df.select_dtypes(exclude = ["object"]).columns
ob_features = [e for e in ob_features if e not in ('region', 'month_of_period_end')]

print("Numerical features : " + str(len(num_features)))
print("Object features : " + str(len(ob_features)))
# df_num = df[ob_features]
# df_ob = df[num_features]

In [ ]:
#Seperate months and year
df_year=df['month_of_period_end'].str.split("-", n=2, expand=True)

df_year['year']=df_year[0].astype('int64')

df_year['year'] = df_year['year'].apply(lambda x: x+2000)

df_year.drop(0,axis=1,inplace=True)

df_year.rename(columns={1:'month'},inplace=True)
print(df_year.head())

#add the year and date back to the dataframe 
df=pd.concat([df_year,df], axis=1)

In [ ]:
df.head(3)

In [ ]:
# change the order fo the columns 
#attribute: https://stackoverflow.com/questions/13148429/how-to-change-the-order-of-dataframe-columns
def change_column_order(df, col_name, index):
    cols = df.columns.tolist()
    cols.remove(col_name)
    cols.insert(index, col_name)
    return df[cols]
df = change_column_order(df, 'region', 0)
df.head(2)

In [3]:
# Prepare object columns for conversion and convert
df[ob_features] = df[ob_features].replace({'\$': '','%': '', 'K': '',',':''}, regex=True)
for col in ob_features:
    df[col]=df[col].astype('float32',)
df.info(5)

NameError: name 'df' is not defined

In [4]:
df.head(3)

NameError: name 'df' is not defined

In [16]:
df=df.round(decimals=3)

In [17]:
#after checking that the date matches .. drop "month_of_period_end"
df=df.drop('month_of_period_end',axis=1)

In [18]:
def eda(dataframe): #Credit: Ritika Bhasker, DSI alum
    print ("missing values \n", dataframe.isnull().sum())
    print ("dataframe index \n", dataframe.index)
    print ("dataframe types \n", dataframe.dtypes)
    print ("dataframe shape \n", dataframe.shape)
    print ("dataframe describe \n", dataframe.describe())
    for item in dataframe:
        print (item)
        print (dataframe[item].nunique())
eda(df)


missing values 
 region                      0
month                       0
year                        0
median_sale_price           0
median_sale_price_mom       0
median_sale_price_yoy       0
homes_sold                  0
homes_sold_mom              0
homes_sold_yoy              0
new_listings                0
new_listings_mom            0
new_listings_yoy            0
inventory                   0
inventory_mom               0
inventory_yoy               0
days_on_market              0
days_on_market_mom          0
days_on_market_yoy          0
average_sale_to_list        0
average_sale_to_list_mom    0
average_sale_to_list_yoy    0
dtype: int64
dataframe index 
 Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            2902, 2903, 2904, 2905, 2906, 2907, 2908, 2909, 2910, 2911],
           dtype='int64', length=2904)
dataframe types 
 region                       object
month                        object
year                          in

In [19]:
# save to pickle 
import pickle 
df.to_pickle("./assets/housing_data.pkl")

In [20]:
export = ['region', 'month', 'year','median_sale_price','homes_sold','homes_sold_mom','days_on_market','average_sale_to_list_mom']
df_house_for_merge=df[export]

In [21]:
df_house_for_merge.shape

(2904, 8)

In [22]:
df_house_for_merge.to_pickle("./assets/housing_data_merge.pkl")

In [3]:
df = pd.read_pickle("./assets/housing_data_merge.pkl")

In [63]:
df.set_index('region', inplace=True)

In [68]:
df.drop(['Washington, DC metro area', 'Washington, DC - Northwest Washington', 'Washington, DC - Southwest Washington',
         'Washington, DC - Southeast Washington', 'Washington, DC - Northeast Washington', ''], inplace=True)

In [71]:
df.reset_index(inplace=True)

In [94]:
psa = {'Washington, DC - Trinidad-Langston': '506',
       'Washington, DC - Greenway / Benning / Dupont Park / River Terrace': '507',
       'Washington, DC - Douglas / Shipley Terrace': '502',
       'Washington, DC - American University Park / Friendship Heights / Tenleytown': '205',
       'Washington, DC - Van Ness / Forest Hills / Wakefield': '205',
       'Washington, DC - U-Street': '205',
       'Washington, DC - Brightwood Park / Crestwood / Petworth': '404',
       'Washington, DC - Foggy Bottom / George Washington University / West End': '207',
       'Washington, DC - Woodridge / Fort Lincoln': '503', 
       'Washington, DC - Brentwood / Brookland / Langdon': '505',
       'Washington, DC - Woodridge-Fort Lincoln': '503',
       'Washington, DC - Bloomingdale / Eckington / Edgewood / Truxton Circle': '502',
       'Washington, DC - Woodridge-Fort Lincoln': '503',
       'Washington, DC - Glover Park': '205', 
       'Washington, DC - Bloomingdale / Eckington / Edgewood / Truxton Circle': '502',
       'Washington, DC - Hillcrest / Fairfax Village / Naylor Gardens / Summit Park': '208',
       'Washington, DC - Glover Park / Cathedral Heights / McLean Gardens': '205',
       'Washington, DC - Michigan Park / University Heights': '306',
       'Washington, DC - Logan Circle-Shaw': '307',
       'Washington, DC - Kalorama': '208',
       'Washington, DC - Cathedral-Wesley Heights-McLean Gardens': '205',
       'Washington, DC - Brightwood / Manor Park / Takoma': '401',
       'Washington, DC - Union Station / Kingman Park / Stanton Park': '103',
       'Washington, DC - Benning Heights / Capitol View / Marshall Heights': '507',
       'Washington, DC - Hillcrest-Fairfax Village': '606',
       'Washington, DC - Southeast Chevy Chase': '201',
       'Washington, DC - Hillbrook / Mayfair / Mahaning Heights': '205',
       'Washington, DC - Columbia Heights / Mount Pleasant / Park View': '208',
       'Washington, DC - Cleveland Park / Woodley Park / Embassy Row': '204',
       'Washington, DC - Berry Farm / Buena Vista / Sheridan': '402',
       'Washington, DC - Southwest Ballpark-Navy Yard': '107',
       'Washington, DC - 16th Street Heights-Crestwood-Brightwood Park': '207',
       'Washington, DC - Deanwood': '602',
       'Washington, DC - Adams Morgan / Kalorama Heights / Lanier Heights': '303',
       'Washington, DC - LeDroit Park-Bloomingdale': '306',
       'Washington, DC - Kingman Park': '103',
       'Washington, DC - Chevy Chase-DC': '201',
       'Washington, DC - Colonial Village / Sheperd Park': '404',
       'Washington, DC - Howard University / Le Droit Park': '306',
       'Washington, DC - H Street-NoMa': '502',
       'Washington, DC - Downtown-Penn Quarter-Chinatown': '208',
       'Washington, DC - Foxhall-Palisades': '205',
       'Washington, DC - Bellevue / Congress Heights / Washington Highlands': '204',
       'Washington, DC - Fairlawn / Twining / Randle Highlands / Penn Branch': '607',
       'Washington, DC - Foggy Bottom-Gwu-West End': '207',
       'Washington, DC - Fort Totten-Riggs Park': '405',
       'Washington, DC - River Terrace-Lily Ponds-Mayfair': '202',
       'Washington, DC - Trinidad / Arboretum / Ivy City': '503',
       'Washington, DC - Chinatown / Mount Vernon Square / Penn Quarter': '102',
       'Washington, DC - Foxhall / Palisades / Spring Valley / Wesley Heights': '205', 
       'Washington, DC - Fort Totten / Lamont Riggs / Pleasant Hill / Queens Chapel': '208',
       'Washington, DC - Takoma': '401',
       'Washington, DC - Navy Yard': '107'}

In [101]:
df.region = df.region.map(psa)

In [104]:
df.rename(columns={'region': 'PSA'}, inplace=True)

In [105]:
# df.to_csv('./assets/housing_data_psa.csv')

In [109]:
df.shape

(2644, 8)